# Building Interactive Visualizations in Jupyter Notebook


## What's Jupyter Notebook?
Jupyter Notebook (formerly IPython notebook) is a web application for scientific computation. jupyter supports over 40 languages (called kernals) including R and Matlab. Jupyter is growing in popularity and has become a standard tool for data scientists and researchers.

Python has a wealth of really fantastic libraries for analyzing and plotting data, including matplotlib, numpy and pandas. However, the plots are generated as static images. Wouldn't it be more interesting have dynamically interact with the plots?


## Sample Data
Sample data was downloaded using the command:

```
bokeh sampledata
```

We'll use the [Population by Country 1964 to 2013](/edit/data/gapminder_population.csv)

## Visualizations in Jupyter Notebook
For this demonstration, we'll use the [Bokeh](http://bokeh.pydata.org/en/latest/) and [Plotly](https://plot.ly/) libraries

In [3]:
from bokeh.plotting import output_notebook

output_notebook()

Loading BokehJS ...

## Bokeh Widget
Widgets provide User Interface (UI) components such as sliders and buttons, making your bokeh visualizations dynamic

In [ ]:
from bokeh import w